In [261]:
# Note to import from .py files, must follow structure
# from <.py filename excluding '.py'> import <class name>
# Optionslam creds: aspringfastlaner Options2018

# Importing necessary models
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import seaborn as sns
import matplotlib.dates as dates
# import matplotlib.ticker as ticker
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import sched, time
import pandas_datareader as datareader
from pandas_datareader.data import Options
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import urllib.request as urlreq

from py_vollib.black_scholes import implied_volatility
'''
Calculate the Black-Scholes implied volatility.

Parameters:	
price (float) – the Black-Scholes option price
S (float) – underlying asset price
K (float) – strike price
t (float) – time to expiration in years
r (float) – risk-free interest rate
flag (str) – ‘c’ or ‘p’ for call or put.
>>> S = 100
>>> K = 100
>>> sigma = .2
>>> r = .01
>>> flag = 'c'
>>> t = .5
>>> price = black_scholes(flag, S, K, t, r, sigma)
>>> iv = implied_volatility(price, S, K, t, r, flag)
'''
%matplotlib inline

# Alpha Vantage API Key
# 5HZEUI5AFJB06BUK

# ts = TimeSeries(key='5HZEUI5AFJB06BUK', output_format='pandas')
# data, meta_data = ts.get_intraday(symbol='MSFT',interval='1min', outputsize='full')
# data['close'].plot()
# plt.title('Intraday Times Series for the MSFT stock (1 min)')
# For intraday
# https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=MSFT&interval=1min&apikey=d5HZEUI5AFJB06BUK&datatype=csv

# For daily
# https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=5HZEUI5AFJB06BUK&datatype=csv

In [231]:
'''
Pulling S&P 500 Names
'''

def pull_sp500_list():
    site = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    table = soup.find_all('table')[0]

    tickers = []
    names = []
    gics = []

    # Looping through the soup lxml text table format
    # and splitting each row as a individual string
    # and parsing string to retrieve the date,
    # open, and close information.
    i = 1
    for row in table.find_all('tr'):
        if i == 1:
            i += 1
            continue
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        # row_items is parsed string for each current row where each
        ticker = individual_row[1].split('">')[-1].split('<')[0]
        tickers.append(ticker)
        name = individual_row[2].split('">')[-1].split('<')[0]
        names.append(name)
        gic = individual_row[4].split('>')[1].split('<')[0]
        gics.append(gic)

    sp500 = pd.DataFrame({'Name': names, 'GIC': gics}, index = tickers)
    sp500.index.name = 'Tickers'
    return sp500

nasdaq = pd.read_csv('http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NASDAQ&render=download', index_col = 0)[['Name','LastSale','IPOyear','Sector']]
nyse = pd.read_csv('http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NYSE&render=download', index_col = 0)[['Name','LastSale','IPOyear','Sector']]
us_stocks = pd.concat([nyse,nasdaq], axis = 0).drop_duplicates()
us_stocks = us_stocks[us_stocks['LastSale'] != 'n/a']

In [262]:
'''
Function for pulling implied volatility from option slam for single ticker
'''

def optionslam_scrape(ticker):
    site = 'https://www.optionslam.com/earnings/stocks/' + ticker
    res = requests.get(site)
    soup = bs(requests.get(site).text, "lxml")
    soup = soup.prettify()
    earnings_dict = {'Ticker': ticker}
    
    # Check if there's weekly options
    curr7_implied = "Current 7 Day Implied Movement:"
    implied_move_weekly = "Implied Move Weekly:"
    nextearnings = "Next Earnings Date:"
    if curr7_implied not in soup:
        return 'No Weeklies'
    
    # Parsing if weekly options exist
    # Next earnings date and before or after
    earnings_start_string = "Next Earnings Date:"
    earnings_end_string = '</font>'
    raw_earnings_string = (soup.split(earnings_start_string))[1].split(earnings_end_string)[0].replace('\n','').strip()
    
    try:
        earnings_date = str((raw_earnings_string.split('<b>'))[1].split('<font size="-1">')).split("'")[1].strip()
    except:
        return 'Error Parsing'
    
    earnings_time = str(raw_earnings_string[-2:].strip()).strip()
    
    earnings_dict['Date'] = earnings_date
    earnings_dict['Earnings Time'] = earnings_time
    
    # Parsing 7 day implied move if weekly option exists
    ending_string = '<font size="-2">'
    curr_7 = (soup.split(curr7_implied))[1].split(ending_string)[0].replace('\n','').strip("").split("<td>")[-1].strip()
    earnings_dict['Current 7 Day Implied'] = curr_7
    
    # Parsing Weekly Implied move if weekly option exists
    if implied_move_weekly in soup:
        weekly_implied = (soup.split(implied_move_weekly))[1].split(ending_string)[0].replace('\n','').strip("").split("<td>")[-1].strip()
    else:
        weekly_implied = ''
    earnings_dict["Implied Move Weekly"] = weekly_implied
    
    return earnings_dict

'''
Functions for pulling options data from yahoo Input is a string. The output is a dataframe of the latest
data from yahoo finance tagged with the current date-time. Output columns are pull date-time,
contract name, strike, last price, bid, ask volume, open interest, and IV (in decimal form).
'''
# Function for initial querying of yahoo data
def yahoo_option_query(ticker, unix_date):
    # dt.datetime.fromtimestamp(1525996800).date()
    if unix_date == 'None':
        yahoo_query = 'https://query1.finance.yahoo.com/v7/finance/options/{0}'.format(ticker)
    else:
        yahoo_query = 'https://query1.finance.yahoo.com/v7/finance/options/{0}?date={1}'.format(ticker,str(unix_date))
        
    response = urlreq.urlopen(yahoo_query)
    data = json.loads(response.read().decode())['optionChain']['result'][0]
    
    dict_lst = []
    for key in data.keys():
        dict_lst.append(data[key])
        
    expiries = dict_lst[1]
    strikes = dict_lst[2]
    underlying = dict_lst[4]
    calls = dict_lst[5][0]['calls']
    puts = dict_lst[5][0]['puts']
    
    return (expiries, strikes, underlying, calls, puts)

# Function for creating dataframe for options contracts for a specific maturity date
def create_contract_df(option_dict_lst, strikes):
    df = pd.DataFrame(columns = ['lastPrice','volume','openInterest','bid','ask','mid','impliedVolatility','expiration'],
                      index = strikes)
    for contract in option_dict_lst:
        for col in df.columns:
            if col == 'expiration':
                df.loc[contract['strike'], col] = (dt.datetime.fromtimestamp(contract['expiration']) - dt.datetime.today()).days
            elif col == 'mid':
                df.loc[contract['strike'], col] = (contract['ask'] + contract['bid'])/2
            else:
                df.loc[contract['strike'], col] = contract[col]
    return df.dropna()

# Function for creating straddle view of options for a specific date
def option_chain(calls, puts, strikes):
    call_contracts = create_contract_df(calls, strikes)
    put_contracts = create_contract_df(puts, strikes)
    return call_contracts.join(put_contracts, how = 'inner', lsuffix='_c', rsuffix='_p')

# Function for getting full option data for a specific ticker
def pull_options(ticker):
    initial_near_contract = yahoo_option_query(ticker, 'None')
    
    expiries = initial_near_contract[0]
    options_list = [option_chain(initial_near_contract[3], initial_near_contract[4], initial_near_contract[1])]
    
    for expiry in expiries[1:]:
        next_contract = yahoo_option_query(ticker, expiry)
        options_list.append(option_chain(next_contract[3], next_contract[4], next_contract[1]))
        
    return pd.concat(options_list, axis = 0)
    
def fundamentals(ticker):
    
    site = 'https://finance.yahoo.com/quote/{0}?p={0}'.format(ticker)

    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    table = soup.find_all('table')[1]
    sum_dict = {}

    # Looping through the soup lxml text table format
    # and splitting each row as a individual string
    # and parsing string to retrieve the date,
    # open, and close information.


    for row in table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')[0]

        # row_items is parsed string for each current row where each
        # item in list is the date, open, high, low, close, and volume
        row_items = individual_row.split('<span data-reactid=')[1].split('"><!-- react-text: ')
        if len(row_items) > 1:
            sum_item = row_items[0].split('>')[1].split('<')[0]
            sum_value = row_items[1].split('-->')[1].split('<')[0]
        elif 'YIELD' in row_items[0]:
            try:
                temp_val = row_items[0].split('-value">')[1].split("</td>")[0]
                div_amount = float(temp_val.split(' ')[0])
                div_yield = float(temp_val.split(' ')[1].replace('(','').replace(')','').replace('%',''))

                sum_dict['Div'] = div_amount
                sum_dict['Yield'] = div_yield
            except:
                sum_dict['Div'] = np.nan
                sum_dict['Yield'] = np.nan

        sum_dict[sum_item] = sum_value

    return pd.DataFrame(sum_dict, index = [ticker])

# Function to return fundametal data of a ticker list
def get_fundas(ticker_lst):
    fund_lst = []
    for tick in ticker_lst:
        fund_lst.append(fundamentals(tick))
    return pd.concat(fund_lst,axis = 0)

# Function historical data from alpha advantage
def historical_data(ticker, window = 252, outsize = 'full'):
    alphavantage_link = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={0}&apikey=5HZEUI5AFJB06BUK&datatype=csv&outputsize={1}'.format(ticker, outsize)
    stockframe = pd.read_csv(alphavantage_link, index_col = 0).sort_index()[['open', 'close']]
    stockframe['daily_ret'] = np.log(stockframe['close']/stockframe['close'].shift(1))
    stockframe['intra_ret'] = np.log(stockframe['close']/stockframe['open'])
    stockframe['ovrnt_ret'] = np.log(stockframe['open']/stockframe['close'].shift(1))
    stockframe['daily_vol'] = stockframe.daily_ret.rolling(window=20,center=False).std()
    stockframe['intra_vol'] = stockframe.intra_ret.rolling(window=20,center=False).std()
    stockframe['ovrnt_vol'] = stockframe.ovrnt_ret.rolling(window=20,center=False).std()
    stockframe['daily_ann'] = stockframe.daily_vol*np.sqrt(252)
    stockframe['intra_ann'] = stockframe.intra_vol*np.sqrt((24/6.5)*252)
    stockframe['ovrnt_ann'] = stockframe.ovrnt_vol*np.sqrt((24/17.5)*252)
    stockframe['oc_diff'] = stockframe.close - stockframe.open
    stockframe['daily_dollar_vol'] = stockframe.daily_vol*stockframe.close.shift(1)
    stockframe['daily_dollar_std'] = np.abs(stockframe.oc_diff/stockframe.daily_dollar_vol)

    return stockframe.tail(window)

# Function for building a dataframe of volatilities
# Daily, Intraday, Overnight
def current_volatility(ticker_list):
    
    rows = []
    for tick in ticker_list:
        try:
            curr_vol = historical_data(tick, window = 1, outsize = 'compact')[['daily_ann','intra_ann','ovrnt_ann']]
            curr_vol.index.name = 'Tickers'
            curr_vol.index = [tick]
            rows.append(curr_vol)
        except:
            continue
        
    return pd.concat(rows, axis = 0)

# Function for pulling S&P500 data and calculating volatilities
def sp500_filter():
    sp500 = pull_sp500_list()
    sp500_vols = current_volatility(sp500.index.tolist())
    df = pd.concat([sp500_vols, sp500], axis = 1).dropna()
    
    return df

In [275]:
start = 0
end = 100
us_lst =[]
us_ticker_lst = [string.replace(' ','') for string in us_stocks.index.tolist()]

while end <= len(us_stocks):
    us_lst.append(current_volatility(us_ticker_lst[start:end]))
    if end == len(us_stocks):
        break
    start = end
    end += 100
    if end > len(us_stocks):
        end = len(us_stocks)



C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:164: RuntimeWarning: divide by zero encountered in log


ValueError: No objects to concatenate

In [281]:
us_vol = pd.concat(us_lst, axis = 0)

In [ ]:
sp500_len = len(sp500)
end_range = 100
start_range = 0
i = 0
batch_list = []
while sp500_len > 100:
    tick_lst = sp500[start_range:end_range].index.tolist()
    tickers = str(tick_lst).replace('[','').replace(']','').replace("'","").replace(" ","")
    alphavantage_link = 'https://www.alphavantage.co/query?function=BATCH_STOCK_QUOTES&symbols={0}&apikey=5HZEUI5AFJB06BUK&datatype=csv'.format(tickers)
    batch_list.append(pd.read_csv(alphavantage_link, index_col = 0)[['price']])
    start_range = end_range
    end_range = end_range + 100
    sp500_len = sp500_len - 100
    i += 1
start_range = i*100
tick_lst = sp500[start_range:(start_range + sp500_len)].index.tolist()
tickers = str(tick_lst).replace('[','').replace(']','').replace("'","").replace(" ","")
alphavantage_link = 'https://www.alphavantage.co/query?function=BATCH_STOCK_QUOTES&symbols={0}&apikey=5HZEUI5AFJB06BUK&datatype=csv'.format(tickers)
batch_list.append(pd.read_csv(alphavantage_link, index_col = 0)[['price']])
batch_quotes = pd.concat(batch_list, axis = 0)
sp500_df = pd.concat([sp500, batch_quotes], axis = 1)
sp500_df['Intra_Daily_Ratio'] = sp500_df['intra_ann']/sp500_df['daily_ann']
sp500_df = sp500_df.sort_values(['Intra_Daily_Ratio'], ascending = False)

In [233]:
tickers = ['HD', 'CSCO', 'NTES', 'CRM', 'RL', 'WMT', 'DE']
options_1week = {}
for ticker in tickers:
    options_1week[ticker] = pull_options(ticker)

In [240]:
week2 = pd.read_csv('earnings_5-21-2018.csv')
options_2week = {}
for ticker in week2['Tickers'].tolist():
    options_2week[ticker] = pull_options(ticker)

In [283]:

from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *


In [284]:
def get_raw_data(ticker):
    tape = Options(ticker, 'yahoo')
    data = tape.get_all_data()
    return data

In [285]:
dat = get_raw_data('NVDA')

In [ ]:
# FB NVDA

In [290]:
dat[dat.index.get_level_values(1) <= dt.datetime(2018,6,30)]

Last     Bid     Ask        Chg  \
Strike Expiry     Type Symbol                                                   
80.0   2018-06-15 call NVDA180615C00080000  163.00  152.70  155.50   0.000000   
                  put  NVDA180615P00080000    0.01    0.01    0.06   0.000000   
85.0   2018-06-15 call NVDA180615C00085000  136.10  129.35  131.10   0.000000   
                  put  NVDA180615P00085000    0.03    0.02    0.08   0.000000   
90.0   2018-06-15 call NVDA180615C00090000  141.60  143.05  145.60   0.000000   
                  put  NVDA180615P00090000    0.07    0.02    0.10   0.020000   
95.0   2018-06-15 call NVDA180615C00095000  139.35  136.55  137.05   6.700012   
                  put  NVDA180615P00095000    0.10    0.00    0.13   0.010000   
100.0  2018-06-15 call NVDA180615C00100000  152.98  158.90  161.80   0.000000   
                  put  NVDA180615P00100000    0.01    0.00    0.11   0.000000   
105.0  2018-06-15 call NVDA180615C00105000  125.99  125.10  126.35   4.479996   
                  put  NVDA180615P00105000    0.06    0.01    0.07   0.000000   
110.0  2018-06-15 call NVDA180615C00110000  123.10  128.65  129.85   0.000000   
                  put  NVDA180615P00110000    0.05    0.00    0.05   0.000000   
115.0  2018-06-15 call NVDA180615C00115000  118.15  124.00  124.45   0.000000   
                  put  NVDA180615P00115000    0.03    0.00    0.21   0.000000   
120.0  2018-05-18 call NVDA180518C00120000  128.85  138.85  141.60   0.000000   
                  put  NVDA180518P00120000    0.01    0.00    0.01   0.000000   
       2018-06-15 call NVDA180615C00120000  139.40  139.00  141.95  29.669990   
                  put  NVDA180615P00120000    0.04    0.01    0.08   0.000000   
125.0  2018-06-15 call NVDA180615C00125000  101.95  113.70  115.00   0.000000   
                  put  NVDA180615P00125000    0.01    0.00    0.21  -0.050000   
130.0  2018-05-18 call NVDA180518C00130000   99.18   98.20   99.85   0.000000   
                  put  NVDA180518P00130000    0.01    0.00    0.02  -0.010000   
       2018-06-15 call NVDA180615C00130000   88.15   96.65   97.50   0.000000   
                  put  NVDA180615P00130000    0.08    0.00    0.10   0.000000   
135.0  2018-05-18 call NVDA180518C00135000   96.80   93.25   94.90   0.000000   
                  put  NVDA180518P00135000    0.04    0.00    0.05   0.000000   
       2018-06-15 call NVDA180615C00135000  125.08  124.00  126.70  40.480003   
                  put  NVDA180615P00135000    0.03    0.00    0.21   0.020000   
...                                            ...     ...     ...        ...   
340.0  2018-06-15 call NVDA180615C00340000    0.21    0.10    0.35   0.110000   
                  put  NVDA180615P00340000   96.50  112.30  112.95   0.000000   
345.0  2018-05-18 call NVDA180518C00345000    0.35    0.07    0.25   0.000000   
       2018-06-15 call NVDA180615C00345000    0.09    0.15    0.23   0.000000   
                  put  NVDA180615P00345000  120.65  117.40  118.75   0.000000   
350.0  2018-05-18 call NVDA180518C00350000    0.06    0.04    0.07   0.030000   
                  put  NVDA180518P00350000  122.30  117.80  119.70   0.000000   
       2018-06-15 call NVDA180615C00350000    0.11    0.01    0.12   0.030000   
                  put  NVDA180615P00350000  120.00  116.20  118.85   0.000000   
355.0  2018-05-18 call NVDA180518C00355000    0.05    0.03    0.12   0.030000   
                  put  NVDA180518P00355000  124.85  127.90  129.05   0.000000   
       2018-06-15 call NVDA180615C00355000    0.09    0.10    0.21   0.080000   
360.0  2018-05-18 call NVDA180518C00360000    0.04    0.00    0.38   0.010000   
                  put  NVDA180518P00360000  121.68  126.30  127.65   0.000000   
       2018-06-15 call NVDA180615C00360000    0.09    0.00    0.13   0.050000   
                  put  NVDA180615P00360000  118.25  126.20  128.85   0.000000   
365.0  2018-05-18 call NVDA180518C00365000    0.03    0.00    0.11  -0.020000

In [268]:
tick = 'FB'
hist = historical_data(tick)
hist